In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [ ]:
import os
from os import listdir, makedirs
from os.path import join, isfile, isdir
from datetime import datetime
import pandas as pd
from tqdm import tqdm

In [ ]:
root_folder = 'drive/My Drive/LaTrobe/Projects/Accelerometer/OA activity data_La Trobe'
ActiGraph_raw = join(root_folder, 'ActiGraph_raw')
ActiGraph_60s_epoch = join(root_folder, 'ActiGraph_60s_epoch')
activPAL_raw = join(root_folder, 'activPAL_raw')
activPAL_15s_epoch = join(root_folder, 'activPAL_15s_epoch')

## ActiGraph_raw   

(HOA53) Start Date 9/03/2020  
(KOA37) Start Date 19/08/2019

In [ ]:
listdir(ActiGraph_raw)

In [ ]:
keys = [f.split(' ')[0] for f in listdir(ActiGraph_raw) if isfile(join(ActiGraph_raw, f))]
print(len(keys), len(listdir(ActiGraph_raw)))

217 217


In [ ]:
print('KOA', len([a for a in keys if 'KOA' in a]))
print('HOA', len([a for a in keys if 'HOA' in a]))
print('OA', len([a for a in keys if 'HOA' not in a and 'KOA' not in a]))

KOA 83
HOA 46
OA 88


Show captured dates

In [ ]:
from datetime import datetime

def diff_dates(date1, date2):

  diff = date2 - date1

  days, seconds = diff.days, diff.seconds
  hours = days * 24 + seconds // 3600
  minutes = (seconds % 3600) // 60
  seconds = seconds % 60

  return '{}h {}m {}d'.format(hours, minutes, seconds)

# d1 = datetime(2013,1,1, 20, 3, 2)
# d2 = datetime(2013,1,13, 4, 2, 1)
# hours, minutes, seconds = diff_dates(d1, d2)
# print ('{}h {}m {}d days between {} and {}'.format(hours, minutes, seconds, d1, d2))
# print ("Happy programmer's day!")

In [ ]:
def get_actigraph_duration(root_folder, files, date_list):

  for f in tqdm(files, desc=root_folder):

    # begin
    with open(join(root_folder, f)) as fle:
      for i, line in enumerate(fle):
        if i == 11:
          dt_begin = datetime.strptime(line.split(',')[0], '%d/%m/%Y %H:%M:%S.%f')
          dt_begin_formatted = dt_begin.strftime('%Y-%m-%d %H:%M:%S')
          # date_list.append([f.split()[0], dt_formatted])
          # dates[user_type].append(dt)
        elif i > 11:
          break

    # end
    with open(join(root_folder, f), 'rb') as flr:
      flr.seek(-2, os.SEEK_END)
      while flr.read(1) != b'\n':
          flr.seek(-2, os.SEEK_CUR)
      last_line = flr.readline().decode()

      dt_end = datetime.strptime(last_line.split(',')[0], '%d/%m/%Y %H:%M:%S.%f')
      dt_end_formatted = dt_end.strftime('%Y-%m-%d %H:%M:%S')
      # dates[user_type].append(dt)

    difference = diff_dates(dt_begin, dt_end)

    # Create data
    # print(f)
    date_list.append([f.split()[0], dt_begin_formatted, dt_end_formatted, difference])

  return date_list


# dates = {'HOA': [], 'KOA': [], 'OA': []}
date_list_global = []

# for HOA
user_type = 'HOA'
files = [f for f in listdir(ActiGraph_raw) if isfile(join(ActiGraph_raw, f)) and user_type in f]
date_list_global = get_actigraph_duration(ActiGraph_raw, files, date_list_global)

# for KOA
user_type = 'KOA'
files = [f for f in listdir(ActiGraph_raw) if isfile(join(ActiGraph_raw, f)) and user_type in f]
date_list_global = get_actigraph_duration(ActiGraph_raw, files, date_list_global)

# for OA
user_type = 'OA'
files = [f for f in listdir(ActiGraph_raw) if isfile(join(ActiGraph_raw, f)) and 'KOA' not in f and 'HOA' not in f]
date_list_global = get_actigraph_duration(ActiGraph_raw, files, date_list_global)


drive/My Drive/LaTrobe/Projects/Accelerometer/OA activity data_La Trobe/ActiGraph_raw: 100%|██████████| 46/46 [00:00<00:00, 238.80it/s]
drive/My Drive/LaTrobe/Projects/Accelerometer/OA activity data_La Trobe/ActiGraph_raw: 100%|██████████| 83/83 [01:57<00:00,  1.42s/it]
drive/My Drive/LaTrobe/Projects/Accelerometer/OA activity data_La Trobe/ActiGraph_raw: 100%|██████████| 88/88 [02:02<00:00,  1.39s/it]


In [ ]:
len(date_list_global)

46

In [ ]:
actigraph_df = pd.DataFrame(date_list_global, columns=['Participant', 'actigraph_Start_time', 'actigraph_End_time', 'actigraph_Time_difference'])

## ActiGraph 60s

In [ ]:
keys = [f.split(' ')[0] for f in listdir(ActiGraph_60s_epoch) if isfile(join(ActiGraph_60s_epoch, f))]
print(len(keys), len(listdir(ActiGraph_60s_epoch)))

217 217


In [ ]:
print('KOA', len([a for a in keys if 'KOA' in a]))
print('HOA', len([a for a in keys if 'HOA' in a]))
print('OA', len([a for a in keys if 'HOA' not in a and 'KOA' not in a]))

KOA 83
HOA 46
OA 88


## activPAL_raw  

Time conversion: https://stackoverflow.com/questions/1703505/excel-date-to-unix-timestamp  

  
Important:
OA 13 and 20 missing

In [ ]:
listdir(activPAL_raw)

In [ ]:
keys = [f.split('-')[0] for f in listdir(activPAL_raw) if isfile(join(activPAL_raw, f))]
print(len(keys), len(listdir(activPAL_raw)))

193 193


In [ ]:
print('KOA', len([a for a in keys if 'KOA' in a]))
print('HOA', len([a for a in keys if 'HOA' in a]))
print('OA', len([a for a in keys if 'HOA' not in a and 'KOA' not in a]))

KOA 64
HOA 42
OA 87


In [ ]:
def get_activpal_duration(root_folder, files, date_list):

  for f in tqdm(files):

    # begin
    with open(join(root_folder, f)) as fle:
      for i, line in enumerate(fle):
        if i == 2:
          ts_begin = (float(line.split(';')[0].replace('\n', '')) - 25569) * 86400
          dt_begin = datetime.utcfromtimestamp(ts_begin).strftime('%Y-%m-%d %H:%M:%S')
          # date_actipal_list.append([f.split('-')[0], dt])
          # dates_actipal_raw[user_type].append(dt)
        elif i > 2:
          break

    # end
    with open(join(root_folder, f), 'rb') as flr:
      flr.seek(-2, os.SEEK_END)
      while flr.read(1) != b'\n':
          flr.seek(-2, os.SEEK_CUR)
      last_line = flr.readline().decode()

      ts_end = (float(last_line.split(';')[0].replace('\n', '')) - 25569) * 86400
      dt_end = datetime.utcfromtimestamp(ts_end).strftime('%Y-%m-%d %H:%M:%S')
      # dates[user_type].append(dt)

    difference = diff_dates(datetime.strptime(dt_begin, '%Y-%m-%d %H:%M:%S'), datetime.strptime(dt_end, '%Y-%m-%d %H:%M:%S'))

    # Create data
    # print(f)
    date_list.append([f.split('-')[0], dt_begin, dt_end, difference])

  return date_list


# dates_actipal_raw = {'HOA': [], 'KOA': [], 'OA': []}
date_actipal_list = []

# for HOA
user_type = 'HOA'
files = [f for f in listdir(activPAL_raw) if isfile(join(activPAL_raw, f)) and user_type in f]
date_actipal_list = get_activpal_duration(activPAL_raw, files, date_actipal_list)

# for KOA
user_type = 'KOA'
files = [f for f in listdir(activPAL_raw) if isfile(join(activPAL_raw, f)) and user_type in f]
date_actipal_list = get_activpal_duration(activPAL_raw, files, date_actipal_list)

# for OA
user_type = 'OA'
files = [f for f in listdir(activPAL_raw) if isfile(join(activPAL_raw, f)) and 'KOA' not in f and 'HOA' not in f]
date_actipal_list = get_activpal_duration(activPAL_raw, files, date_actipal_list)


100%|██████████| 87/87 [01:51<00:00,  1.28s/it]


In [ ]:
activpal_df = pd.DataFrame(date_actipal_list, columns=['Participant', 'activpal_Start_time', 'activpal_End_time', 'activpal_Time_difference'])

In [ ]:
activpal_df.head()

,Participant,activpal_Start_time,activpal_End_time,activpal_Time_difference
0,HOA1,2019-05-06 14:59:46,2019-05-13 08:48:55,161h 49m 9d
1,HOA10,2019-06-24 13:59:47,2019-07-01 10:06:44,164h 6m 57d
2,HOA11,2019-07-01 13:59:47,2019-07-08 11:09:17,165h 9m 30d
3,HOA12,2019-07-01 15:59:47,2019-07-08 10:44:11,162h 44m 24d
4,HOA13,2019-07-08 16:59:47,2019-07-15 10:12:35,161h 12m 48d


In [ ]:
actigraph_df.head()

,Participant,actigraph_Start_time,actigraph_End_time,actigraph_Time_difference
0,HOA1,2019-05-06 15:00:00,2019-05-13 07:59:59,160h 59m 59d
1,HOA2,2019-05-06 15:00:00,2019-05-13 07:59:59,160h 59m 59d
2,HOA3,2019-05-06 16:00:00,2019-05-13 07:59:59,159h 59m 59d
3,HOA4,2019-05-06 16:00:00,2019-05-13 07:59:59,159h 59m 59d
4,HOA5,2019-05-20 13:00:00,2019-05-27 07:59:59,162h 59m 59d


In [ ]:
activpal_df.shape

(193, 4)

In [ ]:
actigraph_df.shape

(217, 4)

In [ ]:
Both_DFs = pd.merge(activpal_df,actigraph_df, how='outer',on=['Participant'])

In [ ]:
Both_DFs.head(50)

,Participant,activpal_Start_time,activpal_End_time,activpal_Time_difference,actigraph_Start_time,actigraph_End_time,actigraph_Time_difference
0,HOA1,2019-05-06 14:59:46,2019-05-13 08:48:55,161h 49m 9d,2019-05-06 15:00:00,2019-05-13 07:59:59,160h 59m 59d
1,HOA10,2019-06-24 13:59:47,2019-07-01 10:06:44,164h 6m 57d,2019-06-24 14:00:00,2019-07-01 07:59:59,161h 59m 59d
2,HOA11,2019-07-01 13:59:47,2019-07-08 11:09:17,165h 9m 30d,2019-07-01 14:00:00,2019-07-08 07:59:59,161h 59m 59d
3,HOA12,2019-07-01 15:59:47,2019-07-08 10:44:11,162h 44m 24d,2019-07-01 16:00:00,2019-07-08 07:59:59,159h 59m 59d
4,HOA13,2019-07-08 16:59:47,2019-07-15 10:12:35,161h 12m 48d,NaN,NaN,NaN
5,HOA14,2019-07-15 14:59:49,2019-07-22 10:17:40,163h 17m 51d,2019-07-15 15:00:00,2019-07-22 07:59:59,160h 59m 59d
6,HOA15,2019-07-29 14:59:47,2019-08-05 10:53:10,163h 53m 23d,2019-07-29 15:00:00,2019-08-05 07:59:59,160h 59m 59d
7,HOA16,2019-07-29 15:59:47,2019-08-05 10:39:14,162h 39m 27d,2019-07-29 16:00:00,2019-08-05 07:59:59,159h 59m 59d
8,HOA17,2019-07-29 15:59:47,2019-08-05 10:45:15,162h 45m 28d,2019-07-29 17:00:00,2019-08-05 07:59:59,158h 59m 59d
9,HOA18,2019-07-29 16:59:47,2019-07-30 19:22:39,26h 22m 52d,2019-07-29 20:00:00,2019-08-05 07:59:59,155h 59m 59d


In [ ]:
Both_DFs.columns

Index(['Participant', 'activpal_Start_time', 'activpal_End_time',
       'activpal_Time_difference', 'actigraph_Start_time',
       'actigraph_End_time', 'actigraph_Time_difference'],
      dtype='object')

In [ ]:
Both_DFs['activpal_Start_time'] = pd.to_datetime(Both_DFs['activpal_Start_time'], format='%Y-%m-%d %H:%M:%S')
Both_DFs['activpal_End_time'] = pd.to_datetime(Both_DFs['activpal_End_time'], format='%Y-%m-%d %H:%M:%S')
Both_DFs['actigraph_Start_time'] = pd.to_datetime(Both_DFs['actigraph_Start_time'], format='%Y-%m-%d %H:%M:%S')
Both_DFs['actigraph_End_time'] = pd.to_datetime(Both_DFs['actigraph_End_time'], format='%Y-%m-%d %H:%M:%S')

## Overlap

In [ ]:

from datetime import datetime
from collections import namedtuple
Range = namedtuple('Range', ['start', 'end'])

overlap_data = []

for index, row in Both_DFs.iterrows():

  if pd.isna(row['activpal_Start_time']) or pd.isna(row['activpal_End_time']) or pd.isna(row['actigraph_Start_time']) or pd.isna(row['actigraph_End_time']):
    overlap_data.append(None)
    continue

  r1 = Range(start=row['activpal_Start_time'], end=row['activpal_End_time'])
  r2 = Range(start=row['actigraph_Start_time'], end=row['actigraph_End_time'])

  latest_start = max(r1.start, r2.start)
  earliest_end = min(r1.end, r2.end)

  if latest_start > earliest_end:
    overlap_data.append(None)
    continue

  overlap = diff_dates(latest_start, earliest_end)
  overlap_data.append(overlap)

# Combine to dataframe
Both_DFs['Overlap'] = overlap_data

In [ ]:
Both_DFs.to_csv('record_avaiability.csv')

## activPal 15s

In [ ]:
keys = [f.split('-')[0] for f in listdir(activPAL_15s_epoch) if isfile(join(activPAL_15s_epoch, f))]
print(len(keys), len(listdir(activPAL_15s_epoch)))

196 196


In [ ]:
keys

In [ ]:
print('KOA', len([a for a in keys if 'KOA' in a]))
print('HOA', len([a for a in keys if 'HOA' in a]))
print('OA', len([a for a in keys if 'HOA' not in a and 'KOA' not in a]))

# KOA50 events


KOA 65
HOA 42
OA 89
